# Medical Image Fundamentals: Chest X-Ray Analysis

Learn medical image analysis basics using chest X-ray feature data.

## Dataset

100 chest X-ray cases (60 Normal, 40 Pneumonia) with extracted features:
- **Intensity Features**: Mean, standard deviation
- **Texture Features**: Contrast, energy, homogeneity, entropy
- **Anatomical Features**: Lung area, heart width ratio, edge density

## Methods
- Feature distribution analysis
- Statistical testing (t-tests)
- Classification with traditional ML
- ROC curve analysis
- Diagnostic performance metrics

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, classification_report, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("Set2")
%matplotlib inline

print("✓ Setup complete")

## 1. Load and Explore Data

In [ ]:
# Load feature data
df = pd.read_csv("sample_xray_features.csv")

print(f"Dataset shape: {df.shape}")
print("\nDiagnosis distribution:")
print(df["diagnosis"].value_counts())
print(f"\nFeatures: {list(df.columns[2:])}")

df.head(10)

In [ ]:
# Summary statistics by diagnosis
feature_cols = [
    "mean_intensity",
    "std_intensity",
    "contrast",
    "energy",
    "homogeneity",
    "entropy",
    "lung_area_percent",
    "heart_width_ratio",
    "edge_density",
]

print("Summary Statistics:\n")
print("Normal Cases:")
print(df[df["diagnosis"] == "Normal"][feature_cols].describe().round(2))
print("\nPneumonia Cases:")
print(df[df["diagnosis"] == "Pneumonia"][feature_cols].describe().round(2))

## 2. Feature Distribution Analysis

In [ ]:
# Plot feature distributions by diagnosis
fig, axes = plt.subplots(3, 3, figsize=(16, 14))
axes = axes.flatten()

for idx, feature in enumerate(feature_cols):
    # Normal cases
    axes[idx].hist(
        df[df["diagnosis"] == "Normal"][feature],
        bins=15,
        alpha=0.6,
        label="Normal",
        color="green",
        edgecolor="black",
    )
    # Pneumonia cases
    axes[idx].hist(
        df[df["diagnosis"] == "Pneumonia"][feature],
        bins=15,
        alpha=0.6,
        label="Pneumonia",
        color="red",
        edgecolor="black",
    )

    axes[idx].set_xlabel(feature.replace("_", " ").title(), fontsize=10)
    axes[idx].set_ylabel("Count", fontsize=10)
    axes[idx].set_title(f"{feature.replace('_', ' ').title()}", fontsize=11, fontweight="bold")
    axes[idx].legend(fontsize=9)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(
    "Note: Pneumonia cases tend to show higher intensity, lower homogeneity, and reduced lung area."
)

## 3. Statistical Testing

In [ ]:
# Perform t-tests for each feature
normal_data = df[df["diagnosis"] == "Normal"]
pneumonia_data = df[df["diagnosis"] == "Pneumonia"]

print("Statistical Comparison (Independent t-tests):")
print("=" * 80)

results = []
for feature in feature_cols:
    normal_values = normal_data[feature]
    pneumonia_values = pneumonia_data[feature]

    t_stat, p_value = stats.ttest_ind(normal_values, pneumonia_values)

    # Effect size (Cohen's d)
    pooled_std = np.sqrt(
        (
            (len(normal_values) - 1) * normal_values.std() ** 2
            + (len(pneumonia_values) - 1) * pneumonia_values.std() ** 2
        )
        / (len(normal_values) + len(pneumonia_values) - 2)
    )
    cohens_d = (normal_values.mean() - pneumonia_values.mean()) / pooled_std

    sig = (
        "***"
        if p_value < 0.001
        else ("**" if p_value < 0.01 else ("*" if p_value < 0.05 else "ns"))
    )

    print(f"{feature:.<30} t={t_stat:.3f}, p={p_value:.4f} {sig}, d={cohens_d:.3f}")
    results.append(
        {"Feature": feature, "t-statistic": t_stat, "p-value": p_value, "Cohen's d": cohens_d}
    )

print("=" * 80)
print("Significance: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")
print("Effect size: |d| > 0.8 = large, > 0.5 = medium, > 0.2 = small")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values("p-value")

## 4. Feature Importance

In [ ]:
# Visualize effect sizes
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(
    results_df["Feature"],
    np.abs(results_df["Cohen's d"]),
    color=["green" if p < 0.05 else "gray" for p in results_df["p-value"]],
    edgecolor="black",
    alpha=0.7,
)

ax.axvline(0.2, color="orange", linestyle="--", linewidth=1, alpha=0.5, label="Small effect")
ax.axvline(0.5, color="blue", linestyle="--", linewidth=1, alpha=0.5, label="Medium effect")
ax.axvline(0.8, color="red", linestyle="--", linewidth=1, alpha=0.5, label="Large effect")

ax.set_xlabel("Absolute Effect Size (|Cohen's d|)", fontsize=12)
ax.set_title("Feature Discriminative Power", fontsize=14, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis="x")
plt.tight_layout()
plt.show()

print("\nMost Discriminative Features (largest effect sizes):")
print(results_df[["Feature", "Cohen's d", "p-value"]].head(5).to_string(index=False))

## 5. Correlation Analysis

In [ ]:
# Feature correlation matrix
correlation = df[feature_cols].corr()

fig, ax = plt.subplots(figsize=(11, 9))
mask = np.triu(np.ones_like(correlation, dtype=bool), k=1)
sns.heatmap(
    correlation,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    mask=mask,
    square=True,
    ax=ax,
    vmin=-1,
    vmax=1,
    cbar_kws={"label": "Correlation"},
)
ax.set_title("Feature Correlation Matrix", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print("\nHigh Correlations (|r| > 0.7):")
for i in range(len(correlation.columns)):
    for j in range(i + 1, len(correlation.columns)):
        if abs(correlation.iloc[i, j]) > 0.7:
            print(
                f"  {correlation.columns[i]} - {correlation.columns[j]}: r = {correlation.iloc[i, j]:.3f}"
            )

## 6. Classification Model

In [ ]:
# Prepare data
X = df[feature_cols].values
y = (df["diagnosis"] == "Pneumonia").astype(int).values  # 1 = Pneumonia, 0 = Normal

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print("\nClass distribution (training):")
print(f"  Normal: {(y_train == 0).sum()}")
print(f"  Pneumonia: {(y_train == 1).sum()}")

In [ ]:
# Train Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predictions
lr_pred = lr_model.predict(X_test_scaled)
lr_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

rf_pred = rf_model.predict(X_test_scaled)
rf_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

print("Model Training Complete")
print(f"\nLogistic Regression - Training Accuracy: {lr_model.score(X_train_scaled, y_train):.3f}")
print(f"Logistic Regression - Test Accuracy: {lr_model.score(X_test_scaled, y_test):.3f}")
print(f"\nRandom Forest - Training Accuracy: {rf_model.score(X_train_scaled, y_train):.3f}")
print(f"Random Forest - Test Accuracy: {rf_model.score(X_test_scaled, y_test):.3f}")

## 7. Model Evaluation

In [ ]:
# Confusion matrices
lr_cm = confusion_matrix(y_test, lr_pred)
rf_cm = confusion_matrix(y_test, rf_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Logistic Regression confusion matrix
sns.heatmap(
    lr_cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    ax=axes[0],
    xticklabels=["Normal", "Pneumonia"],
    yticklabels=["Normal", "Pneumonia"],
)
axes[0].set_xlabel("Predicted", fontsize=11)
axes[0].set_ylabel("Actual", fontsize=11)
axes[0].set_title("Logistic Regression - Confusion Matrix", fontsize=12, fontweight="bold")

# Random Forest confusion matrix
sns.heatmap(
    rf_cm,
    annot=True,
    fmt="d",
    cmap="Greens",
    ax=axes[1],
    xticklabels=["Normal", "Pneumonia"],
    yticklabels=["Normal", "Pneumonia"],
)
axes[1].set_xlabel("Predicted", fontsize=11)
axes[1].set_ylabel("Actual", fontsize=11)
axes[1].set_title("Random Forest - Confusion Matrix", fontsize=12, fontweight="bold")

plt.tight_layout()
plt.show()

In [ ]:
# Classification reports
print("Logistic Regression Performance:")
print("=" * 60)
print(classification_report(y_test, lr_pred, target_names=["Normal", "Pneumonia"]))

print("\nRandom Forest Performance:")
print("=" * 60)
print(classification_report(y_test, rf_pred, target_names=["Normal", "Pneumonia"]))

## 8. ROC Curve Analysis

In [ ]:
# Calculate ROC curves
lr_fpr, lr_tpr, lr_thresholds = roc_curve(y_test, lr_proba)
lr_auc = auc(lr_fpr, lr_tpr)

rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_test, rf_proba)
rf_auc = auc(rf_fpr, rf_tpr)

# Plot ROC curves
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(
    lr_fpr, lr_tpr, linewidth=2, label=f"Logistic Regression (AUC = {lr_auc:.3f})", color="blue"
)
ax.plot(rf_fpr, rf_tpr, linewidth=2, label=f"Random Forest (AUC = {rf_auc:.3f})", color="green")
ax.plot([0, 1], [0, 1], "k--", linewidth=1, label="Random Classifier (AUC = 0.500)")

ax.set_xlabel("False Positive Rate", fontsize=12)
ax.set_ylabel("True Positive Rate (Sensitivity)", fontsize=12)
ax.set_title("ROC Curves - Pneumonia Detection", fontsize=14, fontweight="bold")
ax.legend(fontsize=11, loc="lower right")
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
plt.tight_layout()
plt.show()

print("ROC-AUC Interpretation:")
print(f"  Logistic Regression AUC: {lr_auc:.3f}")
print(f"  Random Forest AUC: {rf_auc:.3f}")
print("\n  0.90-1.00: Excellent")
print("  0.80-0.90: Good")
print("  0.70-0.80: Fair")
print("  0.60-0.70: Poor")
print("  0.50-0.60: Fail")

## 9. Feature Importance (Random Forest)

In [ ]:
# Get feature importances
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame(
    {"Feature": feature_cols, "Importance": importances}
).sort_values("Importance", ascending=False)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(
    feature_importance_df["Feature"],
    feature_importance_df["Importance"],
    color="forestgreen",
    alpha=0.7,
    edgecolor="black",
)
ax.set_xlabel("Feature Importance", fontsize=12)
ax.set_title("Random Forest Feature Importance", fontsize=14, fontweight="bold")
ax.grid(True, alpha=0.3, axis="x")
plt.tight_layout()
plt.show()

print("\nTop Features for Pneumonia Detection:")
print(feature_importance_df.to_string(index=False))

## 10. Clinical Interpretation

In [ ]:
# Diagnostic performance metrics
def calculate_metrics(cm):
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)  # Positive Predictive Value
    npv = tn / (tn + fn)  # Negative Predictive Value
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return sensitivity, specificity, ppv, npv, accuracy


lr_sens, lr_spec, lr_ppv, lr_npv, lr_acc = calculate_metrics(lr_cm)
rf_sens, rf_spec, rf_ppv, rf_npv, rf_acc = calculate_metrics(rf_cm)

print("=" * 80)
print("DIAGNOSTIC PERFORMANCE SUMMARY")
print("=" * 80)
print("\nLogistic Regression:")
print(
    f"  Sensitivity (Recall): {lr_sens:.3f} - Correctly identifies {lr_sens * 100:.1f}% of pneumonia cases"
)
print(
    f"  Specificity:          {lr_spec:.3f} - Correctly identifies {lr_spec * 100:.1f}% of normal cases"
)
print(
    f"  PPV (Precision):      {lr_ppv:.3f} - {lr_ppv * 100:.1f}% of positive predictions are correct"
)
print(
    f"  NPV:                  {lr_npv:.3f} - {lr_npv * 100:.1f}% of negative predictions are correct"
)
print(f"  Accuracy:             {lr_acc:.3f}")
print(f"  AUC:                  {lr_auc:.3f}")

print("\nRandom Forest:")
print(
    f"  Sensitivity (Recall): {rf_sens:.3f} - Correctly identifies {rf_sens * 100:.1f}% of pneumonia cases"
)
print(
    f"  Specificity:          {rf_spec:.3f} - Correctly identifies {rf_spec * 100:.1f}% of normal cases"
)
print(
    f"  PPV (Precision):      {rf_ppv:.3f} - {rf_ppv * 100:.1f}% of positive predictions are correct"
)
print(
    f"  NPV:                  {rf_npv:.3f} - {rf_npv * 100:.1f}% of negative predictions are correct"
)
print(f"  Accuracy:             {rf_acc:.3f}")
print(f"  AUC:                  {rf_auc:.3f}")
print("=" * 80)

print("\nClinical Interpretation:")
print("  - High sensitivity is crucial to avoid missing pneumonia cases")
print("  - High specificity reduces unnecessary treatment of normal cases")
print("  - PPV indicates how confident we can be in positive diagnoses")
print("  - NPV indicates how confident we can be in negative diagnoses")
print("  - AUC > 0.8 suggests good discriminative ability")

## Key Concepts Learned

### Medical Image Features
- **Intensity**: Brightness patterns in X-rays
- **Texture**: Spatial patterns (homogeneity, entropy)
- **Anatomical**: Organ measurements and proportions

### Statistical Testing
- **t-tests**: Compare features between groups
- **Effect Size**: Magnitude of difference (Cohen's d)
- **p-values**: Statistical significance

### Classification Performance
- **Sensitivity**: True positive rate (recall)
- **Specificity**: True negative rate
- **PPV**: Positive predictive value (precision)
- **NPV**: Negative predictive value
- **AUC**: Overall discriminative ability

### ROC Curves
- Trade-off between sensitivity and specificity
- Threshold selection for clinical application
- AUC as single performance metric

## Next Steps

### Extend the Analysis
- Add more features (shape descriptors, wavelet features)
- Try deep learning (CNNs on actual images)
- Multi-class classification (multiple diseases)
- Ensemble methods

### Real Medical Images
- **[NIH Chest X-Ray Dataset](https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community)**: 100,000+ images
- **[MIMIC-CXR](https://physionet.org/content/mimic-cxr/2.0.0/)**: 377,000+ images with reports
- **[CheXpert](https://stanfordmlgroup.github.io/competitions/chexpert/)**: 224,000+ chest radiographs

### Advanced Methods
- Convolutional Neural Networks (ResNet, DenseNet)
- Transfer learning from ImageNet
- Attention mechanisms
- Explainability (GradCAM, SHAP)
- Multi-task learning

## Resources

- **[RadioGraphics](https://pubs.rsna.org/journal/radiographics)**: Radiology journal
- **[Grand Challenge](https://grand-challenge.org/)**: Medical imaging competitions
- **[PyRadiomics](https://pyradiomics.readthedocs.io/)**: Feature extraction library
- **Textbook**: *Medical Image Analysis* by Atam Dhawan